In [42]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [43]:
import findspark
findspark.init()
findspark.find()
import pyspark
import math

from pyspark import SparkContext, SparkConf

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, IntegerType
from pyspark.sql.functions import col,expr
from pyspark.sql.functions import sum,count
from pyspark.sql.functions import stddev_pop
from pyspark.sql.functions import max
from pyspark.sql.functions import mean ,corr,max, min,stddev_pop ,stddev_samp,covar_pop,covar_samp,variance
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import RFormula
spark = SparkSession.builder.getOrCreate()
sc =SparkContext.getOrCreate()

In [44]:
dataset = spark.read.option("inferschema","true").option("header","true").csv("dataset.csv")
dataset.show(5)

+-------+-----------------------------+---------+
|Student|% achieved in the assignments|Pass exam|
+-------+-----------------------------+---------+
|  Alice|                           70|      Yes|
|    Bob|                           36|       No|
|  Carol|                           95|      Yes|
|    Dan|                           63|      Yes|
|    Eve|                           43|       No|
+-------+-----------------------------+---------+
only showing top 5 rows



In [45]:
dataset.printSchema()

root
 |-- Student: string (nullable = true)
 |-- % achieved in the assignments: integer (nullable = true)
 |-- Pass exam: string (nullable = true)



In [46]:
dataset_mean = dataset.groupBy("Pass exam").agg(mean("% achieved in the assignments"))
dataset_mean.show()

+---------+----------------------------------+
|Pass exam|avg(% achieved in the assignments)|
+---------+----------------------------------+
|       No|                              29.8|
|      Yes|                 76.16666666666667|
+---------+----------------------------------+



In [47]:
dataset_var = dataset.groupBy("Pass exam").agg(variance("% achieved in the assignments"))
dataset_var.show()

+---------+---------------------------------------+
|Pass exam|var_samp(% achieved in the assignments)|
+---------+---------------------------------------+
|       No|                     131.70000000000002|
|      Yes|                     267.76666666666665|
+---------+---------------------------------------+



In [48]:
dataset_yes = dataset.where(col("Pass exam") == "Yes")
dataset_yes.show()

+-------+-----------------------------+---------+
|Student|% achieved in the assignments|Pass exam|
+-------+-----------------------------+---------+
|  Alice|                           70|      Yes|
|  Carol|                           95|      Yes|
|    Dan|                           63|      Yes|
|  Frank|                           84|      Yes|
|  Grace|                           54|      Yes|
|   Judy|                           91|      Yes|
+-------+-----------------------------+---------+



In [49]:
dataset_no = dataset.where(col("Pass exam") == "No")
dataset_no.show()

+-------+-----------------------------+---------+
|Student|% achieved in the assignments|Pass exam|
+-------+-----------------------------+---------+
|    Bob|                           36|       No|
|    Eve|                           43|       No|
|  Heidi|                           15|       No|
|   Ivan|                           21|       No|
|Mallory|                           34|       No|
+-------+-----------------------------+---------+



In [50]:
n = dataset.count()
print(n)

11


In [51]:
n_yes = dataset_yes.count()
print(n_yes)
n_no = dataset_no.count()
print(n_no)

6
5


In [52]:
p_yes = n_yes/n
p_no  = n_no/n
print(p_yes)
print(p_no)

0.5454545454545454
0.45454545454545453


COLLECT IS USED TO TRANSFORM OUR SPARK CODE INTO LIST


GAUSSIAN NAIVE BAYES FORMULA


In [53]:
dataset.groupBy("Pass exam").agg(variance("% achieved in the assignments")).collect()

[Row(Pass exam='No', var_samp(% achieved in the assignments)=131.70000000000002),
 Row(Pass exam='Yes', var_samp(% achieved in the assignments)=267.76666666666665)]

In [58]:
var_n = dataset.groupBy("Pass exam").agg(variance("% achieved in the assignments")).collect()[0][1]
var_y = dataset.groupBy("Pass exam").agg(variance("% achieved in the assignments")).collect()[1][1]
var_y

267.76666666666665

In [65]:
mean_n = dataset.groupBy("Pass exam").agg(mean("% achieved in the assignments")).collect()[0][1]
mean_y = dataset.groupBy("Pass exam").agg(mean("% achieved in the assignments")).collect()[1][1]
mean_y

76.16666666666667

In [66]:
final_p_y = ((1/math.sqrt(2* math.pi*var_y))*math.exp((-0.5 * ((61 - mean_y)**2)/var_y)))
final_p_y

0.015866793996967672

In [67]:
final_p_n = ((1/math.sqrt(2* math.pi*var_n))*math.exp((-0.5 * ((61 - mean_n)**2)/var_n)))
final_p_n

0.0008631916482906215